# tf.function 
是 TensorFlow 中一个非常重要的装饰器，它的作用是 将 Python 中的函数转换为 TensorFlow 的计算图（Graph），从而提升执行效率、支持图模式的优化和部署。

## ✅ 基本用法

In [ ]:
import tensorflow as tf

@tf.function
def add(a, b):
    return a + b

x = tf.constant(2)
y = tf.constant(3)
print(add(x, y))  # 输出：tf.Tensor(5, shape=(), dtype=int32)


和普通的 Python 函数不同，tf.function 会将函数 编译为图模式（Graph Mode），避免解释器开销，使 TensorFlow 更快、更优化。

### 📌 使用场景
训练循环（如 train_step()）\
自定义模型中的 forward 方法（call()）\
性能瓶颈的函数\
导出 SavedModel（部署时）

### ✅ 等价的写法
你也可以这样调用：

In [ ]:
def multiply(a, b):
    return a * b

graph_func = tf.function(multiply)
print(graph_func(tf.constant(4), tf.constant(5)))


## ⚠ 注意事项
1. 不要在函数中使用Python 的 print：

In [ ]:
@tf.function
def f(x):
    print("Tracing...")  # 只会在首次“tracing”时执行
    return x * x
    
使用 tf.print() 代替，它会出现在图执行中。

2. 动态控制流建议使用 TensorFlow 版本的操作，如：\
tf.cond() 代替 if \
tf.while_loop() 代替 while \
虽然现在 tf.function 也能追踪部分 Python 控制流，但图模式中建议使用 TF 控制流操作以确保兼容性。

3. 自动追踪和缓存\
当 tf.function 装饰的函数被调用时，它会根据传入参数类型构建不同的计算图，并缓存起来。不同的参数类型会触发重新 tracing。

#### 🔍 查看 Graph 模式
你可以查看函数构建的 Graph：

In [ ]:
@tf.function
def square(x):
    return x * x

print(square.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.float32)).graph)


#### ✅ 常用于训练步骤中

In [ ]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_fn(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss
